In [2]:
from PyPDF2 import PdfReader

In [3]:
reader = PdfReader('2201.11903.pdf')

In [4]:
print(len(reader.pages))

43


In [5]:
page = reader.pages[1]

In [67]:
def get_full_text(reader):
    full_text=""
    for page in reader.pages:
        text = page.extract_text()
        full_text += "\n" + text
    return full_text

In [68]:
full_text = get_full_text(reader)

In [7]:
from langchain import OpenAI
openai = OpenAI(model_name="text-davinci-003", temperature=0.7)

In [75]:
import re
def extract_section_title(text):
    text = text.split("\n")
    title_sec = {}
    sec_so_far = ""
    prev_title = ""
    title = ""
    for para in text:
        if re.search("^\d+ [A-Za-z  -]+$", para):
            prev_title = title
            title = para
            title_sec[prev_title] = sec_so_far.strip()
            sec_so_far=""
        else:
            sec_so_far += "\n" + para
    return title_sec

In [76]:
sec_text = extract_section_title(full_text)

In [77]:
sec_text.keys()

dict_keys(['', '1 Introduction', '2 Chain-of-Thought Prompting', '3 Arithmetic Reasoning', '4 Commonsense Reasoning', '5 Symbolic Reasoning', '6 Discussion', '7 Related Work'])

In [80]:
from langchain import PromptTemplate


template = """
Generate a bullet style summary for the following text: {section}"""

summary_prompt = PromptTemplate(
    input_variables=["section"],
    template=template,
)



In [81]:
from langchain.chains import LLMChain
from langchain.chains.base import Chain

summary_chain = LLMChain(llm=openai, prompt=summary_prompt)

In [85]:
# Pretty print constants

bold = '\033[1m'
unbold = '\033[0m'

In [95]:
for key, val in sec_text.items():
    if key != "":
        print(f"{bold}{key}{unbold}")
        if len(val.split(" ")) > 1000:
            val = " ".join(val.split(" ")[:1000])
        print(summary_chain.run(section=f"{val}"))
        print("\n")
        print("-" * 80)
        print("\n")

1 Introduction


• This work explores how the reasoning ability of large language models can be unlocked by a simple method motivated by two ideas: (1) generating natural language rationales that lead to the final answer and (2) in-context few-shot learning via prompting. 
• The approach, chain-of-thought prompting, consists of prompting the model with a few input–output exemplars demonstrating the task, along with a chain of thought - a series of intermediate natural language reasoning steps that lead to the final output. 
• Empirical evaluations show that chain-of-thought prompting outperforms standard prompting, sometimes to a striking degree. 
• On the GSM8K benchmark of math word problems, chain-of-thought prompting with PaLM 540B outperforms standard prompting by a large margin and achieves new state-of-the-art performance.
• This work underscores how large language models can learn via a few examples with natural language data about the task.


----------------------------------

In [96]:
"""
Work Pending and to be done during the project
1. Better extraction of section and content
2. Split/merge sections with long subsections
3. Experiment with non-gpt generative models finetuned especially for this task using data collected
4. Prepare a semi-decent UI
5. Add graphics
6. Add style based PPT 
"""